In [ ]:
# train_and_save_model.ipynb (FINAL PRODUCTION VERSION - HYBRID XGBOOST-LSTM)

import pandas as pd
import numpy as np
import os
import joblib
from datetime import timedelta
import warnings
import re

# Deep Learning Libraries
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
# from tensorflow.keras.optimizers import Adam
import tensorflow as tf

# แทนที่ Sequential, Adam และ Layer ต่างๆ
Sequential = tf.keras.models.Sequential
LSTM = tf.keras.layers.LSTM
Dense = tf.keras.layers.Dense
Dropout = tf.keras.layers.Dropout
Input = tf.keras.layers.Input
Adam = tf.keras.optimizers.Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error
import xgboost as xgb # For Classification Step

warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)

# --- Config & Hyperparameters ---
DATA_FILE_PATH = 'data/Training_Data_Final.xlsx'
MODEL_LSTM_PATH = 'models/inventory_lstm_model.h5'
MODEL_XGB_CLASS_PATH = 'models/lead_time_classifier.pkl' # Save XGBoost Classifier
SCALER_X_PATH = 'models/scaler_x.pkl'
SCALER_Y_PATH = 'models/scaler_y.pkl'
METADATA_PATH = 'models/model_metadata.pkl'

TIME_STEPS = 12
EPOCHS = 200
BATCH_SIZE = 32

# Features for LSTM (Include Output from XGBoost Step)
X_features_lstm = [
    'month_num',
    'Patient_Count', 'Patient_E', 'Patient_I', 'Patient_O',
    'Total_SKU_Usage',
    'LT_CATEGORY_PRED',   # <--- Feature from XGBoost Classification
    'Safety_Stock_Qty'
]

os.makedirs('models', exist_ok=True)
os.makedirs('data', exist_ok=True)

print("="*50)
print(" HYBRID SYSTEM SETUP (XGBoost Classification -> LSTM Forecasting)")
print("="*50)

# --- Cell 2: Data Loading & Prep ---

df_train = pd.DataFrame()
base_date = pd.to_datetime('2024-01-01')

try:
    print("\n 1. LOADING DATA...")
    if DATA_FILE_PATH.endswith('.csv'):
        df_raw = pd.read_csv(DATA_FILE_PATH)
    else:
        df_raw = pd.read_excel(DATA_FILE_PATH)

    # *** Show existing columns BEFORE processing ***
    print(f"   Columns found in file: {df_raw.columns.tolist()}")

    df_raw.columns = df_raw.columns.str.strip()

    # Map Columns
    df_raw.rename(columns={
        'Visit_Campus': 'Patient_Count',
        'Min_Stock': 'Safety_Stock_Qty',
    }, inplace=True)

    # UOM Standardization
    print("   Standardizing Units...")
    conv_col = 'Conversion_Factor'
    if conv_col not in df_raw.columns:
         for col in df_raw.columns:
            if re.sub(r'[^a-zA-Z0-9]', '', col).lower() == 'conversionfactor':
                conv_col = col; break

    if conv_col in df_raw.columns:
        df_raw[conv_col] = pd.to_numeric(df_raw[conv_col], errors='coerce').fillna(1)
        df_raw['Usage_Qty'] = pd.to_numeric(df_raw['Usage_Qty'], errors='coerce').fillna(0)
        df_raw['Usage_Qty'] = df_raw['Usage_Qty'] * df_raw[conv_col]

    # Cleansing
    cols_check = ['Date', 'Usage_Qty', 'SKU', 'Lead_Time_Days']
    valid_check = [c for c in cols_check if c in df_raw.columns]
    df_raw = df_raw.dropna(subset=valid_check)
    df_raw = df_raw[df_raw['Usage_Qty'] >= 0]

    # Aggregation
    agg_dict = {'Usage_Qty': 'sum'}
    for c in df_raw.columns:
        if c not in ['Date', 'SKU', 'Usage_Qty']:
            agg_dict[c] = 'first'
    df_raw = df_raw.groupby(['Date', 'SKU'], as_index=False).agg(agg_dict)

    df_raw['Date'] = pd.to_datetime(df_raw['Date'])
    df_raw['month_num'] = df_raw['Date'].dt.month

except Exception as e:
    print(f" Error: {e}")
    raise e

# --- Cell 3: STEP 1 - XGBoost Classification for Lead Time ---

print("\n 2. STEP 1: TRAINING XGBOOST CLASSIFIER (Lead Time Risk)...")

# Create Target Class for XGBoost (0=Fast, 1=Normal, 2=Slow) based on actual Lead Time
# We train this model so it can predict Risk Category even if Lead Time changes or for new items based on other traits
if 'Lead_Time_Days' in df_raw.columns:
    df_raw['LT_CLASS_TARGET'] = pd.cut(
        df_raw['Lead_Time_Days'],
        bins=[-np.inf, 7, 30, np.inf],
        labels=[0, 1, 2],
        right=True
    ).astype(int)

    # Check if there are at least two unique classes for classification
    unique_classes = df_raw['LT_CLASS_TARGET'].nunique()

    if unique_classes < 2:
        print(f"   Warning: Only {unique_classes} unique class(es) found in 'LT_CLASS_TARGET' after binning. Skipping XGBoost Classifier training.")
        print("   'LT_CATEGORY_PRED' will be set to the only available target class (or 0 if no targets were generated).")
        # Assign a default category, which will be the only unique class if one exists, else 0.
        df_raw['LT_CATEGORY_PRED'] = df_raw['LT_CLASS_TARGET'].iloc[0] if not df_raw['LT_CLASS_TARGET'].empty else 0
        # For consistency with the LSTM feature, ensure it's an int.
        df_raw['LT_CATEGORY_PRED'] = df_raw['LT_CATEGORY_PRED'].astype(int)
    else:
        # Features for Classification (Using item inherent properties)
        # Here we use basic numeric features available. In real case, use Item Category, Vendor ID etc.
        # For this demo, we use Unit_Cost and Safety_Stock as proxies for item complexity
        X_class_features = ['Unit_Cost', 'Safety_Stock_Qty']
        X_class = df_raw[X_class_features].fillna(0)
        y_class = df_raw['LT_CLASS_TARGET']

        # Train Classifier
        xgb_classifier = xgb.XGBClassifier(
            objective='multi:softmax',
            num_class=unique_classes, # Dynamically set num_class based on available unique classes
            n_estimators=100,
            random_state=42
        )
        xgb_classifier.fit(X_class, y_class)

        # Predict back onto dataset (Simulation of using the model)
        df_raw['LT_CATEGORY_PRED'] = xgb_classifier.predict(X_class)

        print(f"   Classifier Trained. Accuracy on train set: {xgb_classifier.score(X_class, y_class):.2f}")
        print(f"   'LT_CATEGORY_PRED' feature created and ready for LSTM.")

else:
    raise ValueError("Lead_Time_Days is required for Classification Step.")

# --- Cell 4: STEP 2 - Deep LSTM Forecasting ---

print("\n 3. STEP 2: PREPARING & TRAINING DEEP LSTM...")

# Data Prep
final_item_list = df_raw['SKU'].unique().tolist()
sku_policy_map = df_raw.groupby('SKU')[['Max_Stock']].mean().to_dict('index') if 'Max_Stock' in df_raw.columns else {}
base_date = df_raw['Date'].min(); max_date = df_raw['Date'].max()

# Scale
scaler_x = MinMaxScaler(feature_range=(0, 1))
scaler_y = MinMaxScaler(feature_range=(0, 1))

valid_features_lstm = [f for f in X_features_lstm if f in df_raw.columns]
print(f"   - LSTM Input Features: {valid_features_lstm} + Past Usage")

feature_cols = valid_features_lstm + ['Usage_Qty']
X_sequences = []; y_targets = []

# Normalize
data_vals = df_raw[feature_cols].values
scaled_feats = scaler_x.fit_transform(df_raw[valid_features_lstm])
scaled_target = scaler_y.fit_transform(df_raw[['Usage_Qty']])
df_scaled = pd.DataFrame(np.hstack([scaled_feats, scaled_target]), columns=feature_cols)
df_scaled['SKU'] = df_raw['SKU'].values
df_scaled['Date'] = df_raw['Date'].values

# Sequence Building
print(f"   - Building sequences for {len(final_item_list)} SKUs...")
for sku in final_item_list:
    sku_data = df_scaled[df_scaled['SKU'] == sku].sort_values('Date')
    if len(sku_data) <= TIME_STEPS: continue

    values = sku_data[feature_cols].values
    for i in range(TIME_STEPS, len(values)):
        X_sequences.append(values[i-TIME_STEPS:i])
        y_targets.append(values[i, -1])

X_train = np.array(X_sequences)
y_train = np.array(y_targets)

if len(X_train) == 0:
    raise ValueError("Not enough data for LSTM sequences.")

# Train LSTM
model = Sequential()
model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=64, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=32, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Fit model
history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=0.1, verbose=1)

print("   LSTM Model Trained.")

# --- Cell 4.1: Evaluation (R-squared, MSE, RMSE) ---
print("\n 4. EVALUATING MODEL PERFORMANCE...")

# Predict on Training Set
y_pred_scaled = model.predict(X_train)

# Inverse Transform to get actual values
y_train_actual = scaler_y.inverse_transform(y_train.reshape(-1, 1))
y_pred_actual = scaler_y.inverse_transform(y_pred_scaled)

# 1. Calculate R-squared
r2 = r2_score(y_train_actual, y_pred_actual)

# 2. Calculate Mean Squared Error (MSE)

mse = mean_squared_error(y_train_actual, y_pred_actual)

# 3. Calculate Root Mean Squared Error (RMSE)

rmse = np.sqrt(mse)


print(f"    Training R-squared (R²): {r2:.4f}")
print(f"    Training Mean Squared Error (MSE): {mse:.4f}")
print(f"    Training Root Mean Squared Error (RMSE): {rmse:.4f}")


# --- Cell 5: Save All Artifacts ---

print("\n 5. SAVING HYBRID SYSTEM ARTIFACTS...")

model.save(MODEL_LSTM_PATH)
joblib.dump(xgb_classifier, MODEL_XGB_CLASS_PATH) # Save Classifier
joblib.dump(scaler_x, SCALER_X_PATH)
joblib.dump(scaler_y, SCALER_Y_PATH)

metadata = {
    'base_date': base_date,
    'max_date': max_date,
    'item_list': final_item_list,
    'feature_cols': feature_cols,
    'valid_features_lstm': valid_features_lstm,
    'class_features': X_class_features, # Features used for classification
    'sku_policy_map': sku_policy_map,
    'time_steps': TIME_STEPS
}
joblib.dump(metadata, METADATA_PATH)

print(f"    Saved LSTM: {MODEL_LSTM_PATH}")
print(f"    Saved Classifier: {MODEL_XGB_CLASS_PATH}")
print("\n HYBRID SYSTEM READY.")

 HYBRID SYSTEM SETUP (XGBoost Classification -> LSTM Forecasting)

 1. LOADING DATA...
   Columns found in file: ['Date', 'Visit_Campus', 'SKU', 'Item_Name', 'Usage_Qty', 'Total_SKU_Usage', 'Lead_Time_Days', 'Unit_Cost', 'Patient_E', 'Patient_I', 'Patient_O', 'Min_Stock', 'Max_Stock', 'UOM', 'Conversion_Factor']
   Standardizing Units...

 2. STEP 1: TRAINING XGBOOST CLASSIFIER (Lead Time Risk)...
   'LT_CATEGORY_PRED' will be set to the only available target class (or 0 if no targets were generated).

 3. STEP 2: PREPARING & TRAINING DEEP LSTM...
   - LSTM Input Features: ['month_num', 'Patient_Count', 'Patient_E', 'Patient_I', 'Patient_O', 'Total_SKU_Usage', 'LT_CATEGORY_PRED', 'Safety_Stock_Qty'] + Past Usage
   - Building sequences for 227 SKUs...
Epoch 1/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - loss: 0.0243 - val_loss: 0.0186
Epoch 2/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0166 - val_loss: 0.0147
Epoch 3/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0140 -

    Training R-squared (R²): 0.7255
    Training Mean Squared Error (MSE): 30.1253
    Training Root Mean Squared Error (RMSE): 5.4887

 5. SAVING HYBRID SYSTEM ARTIFACTS...
    Saved LSTM: models/inventory_lstm_model.h5
    Saved Classifier: models/lead_time_classifier.pkl

 HYBRID SYSTEM READY.
